In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout,GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math
import pandas as pd
import numpy as np
import os

AttributeError: type object 'h5py.h5r.Reference' has no attribute '__reduce_cython__'

In [ ]:
old_working_dir = os.getcwd()
root_dir=old_working_dir+'\\omr.v6i.folder'
TRAIN_SAMPLES = 100
VALIDATION_SAMPLES=100
TRAIN_DATA_DIR = root_dir+'\\train'
VALIDATION_DATA_DIR = root_dir + '\\test'
NUM_CLASSES = 5
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)
val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
                        TRAIN_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        seed=12345,
                        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
                        VALIDATION_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        class_mode='categorical')

In [ ]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape = (IMG_WIDTH,IMG_HEIGHT,3))
    
    for layer in base_model.layers[:]:
        layer.trainable = False # Freeze the layers
        
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions  = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    
    return Model(inputs=input, outputs=predictions)
def prepare(filepath):
    IMG_SIZE = 224 
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
model = model_maker()

model.compile(loss='categorical_crossentropy',
              optimizer= tf.keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])
model.summary()


In [ ]:
history=model.fit(train_generator, steps_per_epoch = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE), epochs=50, validation_data = validation_generator, validation_steps = math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

history=model.fit(train_generator, steps_per_epoch = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE), epochs=50, validation_data = validation_generator, validation_steps = math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

history=model.fit(train_generator,
                    steps_per_epoch = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
                    epochs=10,
                    validation_data = validation_generator,
                    validation_steps = math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

In [ ]:
model.save("fang2_mobilenet.h5")

In [ ]:
history

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'], label='Accuracy')
plt.plot(history.history['val_acc'], label='Validation Accuracy')
plt.title('CNN Metrices (Accuracy)')
plt.ylabel('% value')
plt.xlabel('Epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('CNN Metrices(Loss)')
plt.ylabel('% value')
plt.xlabel('Epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

model = load_model('fang2_mobilenet.h5')
def load(path):
    image = tf.keras.preprocessing.image.load_img(path)
    image = tf.image.resize(image, [224,224])
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    np_image = np.array(input_arr).astype('float32')/255
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [ ]:
path = 'opencv_frame_1.png'
np_images=load(path)
prediction = model.predict(np_images)
print("Predictions:")
print(prediction)
print(np.argmax(prediction))


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:

    {}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")